## Tables and Data Source setup

In [1]:
using ODBC
using DataFrames
using DSWB
using Formatting
using URIParser

dsn = "dswb-natgeo" # Redshift esetTable(tableRt, tableType = "RESOURCE_TABLE")ndpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(table)
setTable(tableRt, tableType = "RESOURCE_TABLE")
;

Redshift endpoint set: 2017-11-29T21:09:02


In [2]:
# Time & Structures First
include("../../Lib/Include-Package-v2.1.jl")
;

In [3]:
#TV = timeVariables(2017,10,27,23,59,2017,11,3,23,59)
#TV = weeklyTimeVariables(days=7)
TV = yesterdayTimeVariables()
;

November 28, 2017 06:59:00 to November 28, 2017 16:59:00 Local Time
November 28, 2017 11:59:00 to November 28, 2017 21:59:00 UTC Time


In [10]:
UP = UrlParamsInit()
UP.agentOs = "%"
UP.deviceType = "%"
UP.limitRows = 10000
#UP.limitRows = 250
UP.orderBy = "time"
UP.pageGroup = "News Article"   #productPageGroup
UP.samplesMin = 10
UP.sizeMin = 20000
UP.timeLowerMs = 2000.0
UP.timeUpperMs = 60000.0
UP.urlRegEx = "%"   #localUrl
UP.urlFull = "%"
UP.usePageLoad=false
UP.btView = "$(table)_all_dt_view_prod" #localtable
UP.rtView = "$(tableRt)_all_dt_view_prod" #localtableRt
UrlParamsValidate(UP)

SP = ShowParamsInit()
SP.desktop=false
SP.mobile=false
SP.devView=false
SP.criticalPathOnly=true
SP.debug=false
SP.debugLevel = 0   # Tests use even numbers with > tests, make this an odd number or zero
ShowParamsValidate(SP)
;

In [11]:
toppageurl = query("""\
    select 
        'None' as urlpagegroup,
        CASE WHEN (position('?' in url) > 0) then trim('/' from (substring(url for position('?' in substring(url from 9)) +7))) else trim('/' from url) end as urlgroup
    FROM $(UP.resourceTable)
    where
        "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC)
    group by urlgroup,urlpagegroup
    limit $(UP.limitRows)
 """);    

In [12]:
#save for debug
toppageurlbackup = deepcopy(toppageurl);        

In [13]:
include("../../Lib/URL-Classification-Package-v2.0.jl")

WellKnownHost = wellKnownHostDictionary(SP.debug);
WellKnownPath = wellKnownPathDictionary(SP.debug);


In [14]:
include("../../Lib/URL-Classification-Package-v2.0.jl")

# Debug
toppageurl = deepcopy(toppageurlbackup)

scrubUrlToPrint(toppageurl);
classifyUrl(toppageurl,showProblems=false);        
#

Yourshot to do: Classify /photos
    ("d3qdfnco3bamip.cloudfront.net","d3qdfnco3bamip.cloudfront.net"),
    ("d3qdfnco3bamip.cloudfront.net","d3qdfnco3bamip.cloudfront.net"),
    ("d2xvc2nqkduarq.cloudfront.net","d2xvc2nqkduarq.cloudfront.net"),
    ("www.google.co.th","www.google.co.th"),
    ("bundle.clearstream.tv","bundle.clearstream.tv"),
    ("d3qdfnco3bamip.cloudfront.net","d3qdfnco3bamip.cloudfront.net"),
    ("d3qdfnco3bamip.cloudfront.net","d3qdfnco3bamip.cloudfront.net"),
    ("www.natgeomaps.com","www.natgeomaps.com"),
Yourshot to do: Classify /search
    ("resus.u3.ucweb.com","resus.u3.ucweb.com"),
    ("cdn-stage.actiflex.org","cdn-stage.actiflex.org"),
    ("mpc.nl.mxptint.net","mpc.nl.mxptint.net"),
    ("dpstvy7p9whsy.cloudfront.net","dpstvy7p9whsy.cloudfront.net"),
Yourshot to do: Classify /tags/barefoot
    ("dcc4iyjchzom0.cloudfront.net","dcc4iyjchzom0.cloudfront.net"),
    ("quantcast584928381.s.moatpixel.com","quantcast584928381.s.moatpixel.com"),
    ("reports.me